# Construction Loan Estimator

## Parameters

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
from dateutil.relativedelta import relativedelta
import datetime
from pandas.tseries.offsets import DateOffset
import numpy as np
from math import e

In [16]:
property_cost = 85000
construction_cost = 115000
loan_term = 15
loan_term_months = loan_term * 12
completed_value = 300000
construction_term = 12
down_payment = 40000
inflation = 0.02
closing_costs = 0.05
square_footage = 1750
assessment_rate = 0.07
mill_levy = 95
initial_rent = 1500
initial_rent_year = initial_rent * 12
rent_growth_rate = 0.02
home_appreciation = 0.05
home_appreciation_month = home_appreciation / 12
current_month = datetime.date.today().replace(day=1)
total_loan_amount = property_cost + construction_cost - down_payment
loan_interest_rate = 0.04
loan_rate_monthly = loan_interest_rate / 12

In [3]:
# Create blank table for monthly level data
monthly_data = pd.DataFrame()

# Create Period to offset the index by one, to indicate month and create periods
monthly_data['Overall Month'] = range(1, loan_term_months + construction_term + 2)
monthly_data['Construction Month'] = np.where(monthly_data['Overall Month'] <= construction_term, monthly_data['Overall Month'], 0)
monthly_data['Loan Month'] = np.where(monthly_data['Overall Month'] > construction_term, monthly_data['Overall Month'] - construction_term, 0)

In [4]:
# Calculate home value
monthly_data['Home Value'] = np.where(monthly_data['Overall Month'] <= construction_term, 0, completed_value * (e**(home_appreciation_month * (monthly_data['Loan Month']))))

# Calculate loan value
monthly_data['Loan Value'] = np.where(monthly_data['Loan Month'] != 0, total_loan_amount * (((1 + loan_rate_monthly) ** (loan_term_months)) - ((1 + loan_rate_monthly)**(monthly_data['Loan Month'] - 1))) / ((1 + loan_rate_monthly)**(loan_term_months) - 1), total_loan_amount)

In [5]:
# Calculate disbursement payments in a linear fashion
monthly_data['Construction Disbursements'] = np.where(monthly_data['Overall Month'] <= construction_term, total_loan_amount / construction_term, 0)

In [6]:
# Calculate monthly payments
monthly_data['Mortgage Payment'] = np.where(monthly_data['Overall Month'] > construction_term, total_loan_amount * (loan_rate_monthly * ((1 + loan_rate_monthly)**loan_term_months)) / ((1 + loan_rate_monthly)**loan_term_months - 1), 0)

# Calculate interest payment
monthly_data['Interest Payment'] = np.where(monthly_data['Overall Month'] <= construction_term, monthly_data['Construction Disbursements'].cumsum() * loan_rate_monthly, monthly_data['Loan Value'] * loan_rate_monthly)
monthly_data['Cumulative Interest Payments'] = monthly_data['Interest Payment'].cumsum()

# Calculate principal payments
monthly_data['Principal Payment'] = np.where(monthly_data['Overall Month'] > construction_term, monthly_data['Mortgage Payment'] - monthly_data['Interest Payment'], 0)
monthly_data['Cumulative Principal Payments'] = monthly_data['Principal Payment'].cumsum()

In [21]:
# Calculate property taxes
monthly_data['Property Tax Payment'] = np.where(monthly_data['Overall Month'] > construction_term, (((monthly_data['Home Value'] * assessment_rate) * (mill_levy / 1000)) / 12), 0)
monthly_data['Cumulative Property Taxes'] = monthly_data['Property Tax Payment'].cumsum()

In [23]:
# Calculate total home payments
monthly_data['Total Home Payments'] = monthly_data['Interest Payment'] + monthly_data['Principal Payment'] + monthly_data['Property Tax Payment']
monthly_data['Cumulative Home Payments'] = monthly_data['Total Home Payments'].cumsum() + down_payment

# Calculate equity metrics
monthly_data['Equity Ownership Proportion'] = 1- (monthly_data['Loan Value'] / total_loan_amount)
monthly_data['Equity Value'] = monthly_data['Equity Ownership Proportion'] * monthly_data['Home Value']

In [24]:
pd.set_option("max_rows", 24)
monthly_data

,Overall Month,Construction Month,Loan Month,Home Value,Loan Value,Construction Disbursements,Mortgage Payment,Interest Payment,Cumulative Interest Payments,Principal Payment,Cumulative Principal Payments,Total Home Payments,Cumulative Home Payments,Equity Ownership Proportion,Equity Valye,Equity Value,Property Tax Payment,Cumulative Property Taxes
0,1,1,0,0.000000,160000.000000,13333.333333,0.000000,44.444444,44.444444,0.000000,0.000000,44.444444,40044.444444,0.000000,0.000000,0.000000,0.000000,0.000000
1,2,2,0,0.000000,160000.000000,13333.333333,0.000000,88.888889,133.333333,0.000000,0.000000,88.888889,40133.333333,0.000000,0.000000,0.000000,0.000000,0.000000
2,3,3,0,0.000000,160000.000000,13333.333333,0.000000,133.333333,266.666667,0.000000,0.000000,133.333333,40266.666667,0.000000,0.000000,0.000000,0.000000,0.000000
3,4,4,0,0.000000,160000.000000,13333.333333,0.000000,177.777778,444.444444,0.000000,0.000000,177.777778,40444.444444,0.000000,0.000000,0.000000,0.000000,0.000000
4,5,5,0,0.000000,160000.000000,13333.333333,0.000000,222.222222,666.666667,0.000000,0.000000,222.222222,40666.666667,0.000000,0.000000,0.000000,0.000000,0.000000
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
188,189,0,177,627210.666016,4694.814175,0.000000,1183.500681,15.649381,56473.250074,1167.851300,156473.037125,1531.079925,296556.033363,0.970657,608806.681480,608806.681480,347.579244,43609.746164
189,190,0,178,629829.495897,3526.962875,0.000000,1183.500681,11.756543,56485.006617,1171.744138,157644.781263,1532.531193,298088.564556,0.977956,615945.838088,615945.838088,349.030512,43958.776676
190,191,0,179,632459.260335,2355.218737,0.000000,1183.500681,7.850729,56492.857346,1175.649952,158820.431215,1533.988521,299622.553077,0.985280,623149.385958,623149.385958,350.487840,44309.264516
191,192,0,180,635100.004984,1179.568785,0.000000,1183.500681,3.931896,56496.789242,1179.568785,160000.000000,1535.451934,301158.005011,0.992628,630417.854101,630417.854101,351.951253,44661.215769
